<div style="text-align: right" align="right"><i>Peter Norvig, December 2025</i></div>

# Advent of Code 2025: The AI Edition

I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, and **my** solutions are [**over here**](Advent2025.ipynb). 

In **this** notebook I show some solutions by various AI Large Language Models: Gemini, Claude, and ChatGPT.

In order to understand what's going on here, you'll have to look at the problem descriptions at [**Advent of Code**](https://adventofcode.com/2025).

Each day I'll choose a model and give it the prompt "*Write code to solve the following problem:*" followed by the full text of the AoC Part 1 problem description. Then I'll prompt again with "*There is a change to the specification:*" followed by the AoC Part 2 description. So far the LLMs are doing quite well. 

For brevity, I have removed some of the models' output, such as:
- Prose descriptions of the programs. (In most cases these were aaccurate and thorough!)
- The "__main__"` idiom for running code from the command line
- Test examples to run.

Overall, the models did well, producing code that gives the correct answer in a reasonable run time. Some of the cosde could be improved stylistically.  (But I guess if you're vibe coding and not looking at the code, maybe that doesn't matter.)

# Day 0

I load my [**AdventUtils.ipynb**](AdventUtils.ipynb) and set the`current_year` so I can access my input files with `get_text(day_number)` and can use my `answer` function to verify whether the AI systems get the right answer.

In [1]:
%run AdventUtils.ipynb
current_year = 2025

# [Day 1](https://adventofcode.com/2025/day/1) Gemini 3 Pro

I started with the Gemini 3 Pro Fast model, which produced this following code:

In [2]:
def solve_safe_password(rotations):
    """
    Simulates the safe dial rotations and counts how many times the dial
    lands on 0.

    Args:
        rotations: A list of strings, each representing a rotation (e.g., "L68").

    Returns:
        The total number of times the dial points at 0 after a rotation.
    """
    # The dial starts at 50
    current_position = 50
    zero_count = 0
    
    # The dial has numbers 0 through 99, so all math is modulo 100
    MODULO = 100

    for rotation in rotations:
        # 1. Parse the instruction
        direction = rotation[0]
        # Distance is the rest of the string converted to an integer
        distance = int(rotation[1:])

        # 2. Determine the rotation value
        if direction == 'R':
            rotation_value = distance
        elif direction == 'L':
            # Left rotations subtract from the current position
            rotation_value = -distance
        else:
            # Should not happen based on problem description
            print(f"Error: Unknown direction '{direction}' in rotation: {rotation}")
            continue

        # 3. Calculate the new position and apply Modulo 100
        # The standard Python % operator handles negative numbers correctly
        # for a positive modulus (e.g., (-1 - 99) % 100 == 0).
        new_position = (current_position + rotation_value) % MODULO

        # 4. Check for Zero
        if new_position == 0:
            zero_count += 1
        
        # 5. Update Position
        current_position = new_position
        
        # Optional: Print step-by-step for debugging/verification
        # print(f"Rotation {rotation}: new position = {new_position}, zero_count = {zero_count}")

    return zero_count

A bit verbose and over-commented, but pretty nice. Below I fetch the Day 1 data with `get_text(1)`, split it into lines (as expected by Gemini's `solve_safe_password` function),  and verify that the answer is correct:

In [3]:
rotations = get_text(1).splitlines()

answer(puzzle=1.1, solution=1182, code=lambda: 
       solve_safe_password(rotations))

Puzzle  1.1:   .0013 seconds, answer 1182            ok

For Part 2 I gave Gemini Fast the updated instructions, and again it had a nice analysis and the following code:

In [4]:
import math

def solve_new_safe_password(rotations):
    """
    Simulates the safe dial rotations and counts every time the dial
    points at 0, including during the rotation movement.

    Args:
        rotations: A list of strings, each representing a rotation (e.g., "L68").

    Returns:
        The total number of times the dial points at 0.
    """
    # The dial starts at 50
    current_position = 50
    total_zero_crossings = 0
    MODULO = 100

    for rotation in rotations:
        direction = rotation[0]
        distance = int(rotation[1:])
        
        S = current_position # Start Position
        D = distance         # Distance
        
        crossings_for_this_rotation = 0

        if direction == 'L':
            # --- LEFT ROTATION ---
            # Dial moves from S -> S-1 -> ... -> 0 -> 99 -> ...
            
            # 1. Distance from S to the first 0: S clicks
            # If D is large enough to cover this distance, the first 0 is crossed.
            
            if D >= S:
                # The first crossing (at position 0) is achieved
                crossings_for_this_rotation += 1
                
                # 2. Remaining distance after first 0 crossing
                remaining_distance = D - S
                
                # 3. Subsequent crossings: The dial crosses 0 every 100 clicks
                # after the first one.
                crossings_for_this_rotation += remaining_distance // MODULO
                
        elif direction == 'R':
            # --- RIGHT ROTATION ---
            # Dial moves from S -> S+1 -> ... -> 99 -> 0 -> 1 -> ...
            
            # 1. Distance from S to the first 0: 100 - S clicks
            # (e.g., 50 to 0 is 50 clicks; 99 to 0 is 1 click)
            distance_to_first_zero = MODULO - S
            
            if D >= distance_to_first_zero:
                # The first crossing (at position 0) is achieved
                crossings_for_this_rotation += 1
                
                # 2. Remaining distance after first 0 crossing
                remaining_distance = D - distance_to_first_zero
                
                # 3. Subsequent crossings: The dial crosses 0 every 100 clicks
                crossings_for_this_rotation += remaining_distance // MODULO
        
        # Add the crossings for this rotation to the total
        total_zero_crossings += crossings_for_this_rotation

        # Calculate the final position to set up for the next rotation
        if direction == 'R':
            rotation_value = distance
        else: # 'L'
            rotation_value = -distance
            
        current_position = (current_position + rotation_value) % MODULO

    return total_zero_crossings

Note that the code does `import math` but then never uses the math module. (I've done that, but I try to eventually clean things up. Maybe that's the different between Gemini "Fast" and Gemini "Thinking" models.)

Here I check whether the code is correct:

In [5]:
answer(1.2, 6907, lambda: 
       solve_new_safe_password(rotations))

Puzzle  1.2:   .0018 seconds, answer 7509            WRONG; expected answer is 6907

Too bad. I switched to the Gemini 3 Pro "Thinking" model and gave the prompt "*That's not quite right.*" It responded with

>It seems I made a subtle logic error in my previous calculation for the Right (R) rotation.
>
>When the dial is at 0, the distance to the *next* zero is **100**, not 0.

which is exactly the error I made on *my* first attempt. Gemini produced the following updated code:

In [6]:
def solve_safe_password_method_2(rotations):
    """
    Simulates the safe dial rotations and counts every time the dial
    points at 0 (Method 0x434C49434B).
    
    Correctly handles cases where the dial starts at 0.
    """
    current_position = 50
    total_zero_crossings = 0
    MODULO = 100

    for rotation in rotations:
        direction = rotation[0]
        distance = int(rotation[1:])
        
        # Determine distance required to hit the *next* 0
        if direction == 'L':
            if current_position == 0:
                dist_to_first_zero = 100
            else:
                dist_to_first_zero = current_position
                
        elif direction == 'R':
            if current_position == 0:
                dist_to_first_zero = 100
            else:
                dist_to_first_zero = 100 - current_position

        # Check if the rotation distance is enough to reach the first zero
        if distance >= dist_to_first_zero:
            # We hit 0 at least once
            count = 1
            
            # Remove the distance traveled to the first zero
            remaining_distance = distance - dist_to_first_zero
            
            # Add any subsequent full rotations (every 100 clicks hits 0 again)
            count += remaining_distance // 100
            
            total_zero_crossings += count

        # Update position for next step
        if direction == 'R':
            current_position = (current_position + distance) % 100
        else:
            current_position = (current_position - distance) % 100

    return total_zero_crossings

This second try is indeed correct:

In [7]:
answer(1.2, 6907, lambda: 
       solve_safe_password_method_2(rotations))

Puzzle  1.2:   .0024 seconds, answer 6907            ok

# [Day 2](https://adventofcode.com/2025/day/2) Claude Opus 4.5

I gave Claude the instructions for Day 2 Part 1 and it wrote some code that produces the correct answer but prints a lot of unneccessary debugging output along the way. I prompted it to "*Change the code to not print anything, just return the answer*" and got the following:

In [8]:
def find_invalid_ids_in_range(start, end):
    invalid_ids = []
    min_digits = len(str(start))
    max_digits = len(str(end))
    
    for total_digits in range(min_digits, max_digits + 1):
        if total_digits % 2 != 0:
            continue
        
        half_digits = total_digits // 2
        pattern_start = 1 if half_digits == 1 else 10 ** (half_digits - 1)
        pattern_end = 10 ** half_digits - 1
        for pattern in range(pattern_start, pattern_end + 1):
            pattern_str = str(pattern)
            invalid_id = int(pattern_str + pattern_str)
            if start <= invalid_id <= end:
                invalid_ids.append(invalid_id)
    
    return invalid_ids


def solve(input_str):
    input_str = input_str.strip().replace('\n', '').replace(' ', '').rstrip(',')
    total = 0
    for r in input_str.split(','):
        if r:
            start, end = map(int, r.split('-'))
            total += sum(find_invalid_ids_in_range(start, end))
    return total

This code is overall rather nice, but conspicously lacks comments and doc strings. It uses the more efficient "enumerate over the first half of the digit string" strategy, but is not precise in narrowing down the range it enumerates over. For example, for the range "999000-109000", this code will enumerate the range (100, 1000), when it could enumerate just the range (999, 1000).

I verified that the code gives the correct answer:

In [9]:
id_ranges = get_text(2)

answer(2.1, 23560874270, lambda:
       solve(id_ranges))

Puzzle  2.1:   .1263 seconds, answer 23560874270     ok

When given the Part 2 instructions, Claude wrote the following code:

In [10]:
def find_invalid_ids_in_range(start, end):
    invalid_ids = set()
    min_digits = len(str(start))
    max_digits = len(str(end))
    
    for total_digits in range(min_digits, max_digits + 1):
        # Pattern length can be 1 to total_digits//2 (need at least 2 repetitions)
        for pattern_len in range(1, total_digits // 2 + 1):
            if total_digits % pattern_len != 0:
                continue
            
            repetitions = total_digits // pattern_len
            if repetitions < 2:
                continue
            
            pattern_start = 1 if pattern_len == 1 else 10 ** (pattern_len - 1)
            pattern_end = 10 ** pattern_len - 1
            
            for pattern in range(pattern_start, pattern_end + 1):
                pattern_str = str(pattern)
                invalid_id = int(pattern_str * repetitions)
                if start <= invalid_id <= end:
                    invalid_ids.add(invalid_id)
    
    return invalid_ids


def solve(input_str):
    input_str = input_str.strip().replace('\n', '').replace(' ', '').rstrip(',')
    total = 0
    for r in input_str.split(','):
        if r:
            start, end = map(int, r.split('-'))
            total += sum(find_invalid_ids_in_range(start, end))
    return total

Claude's code produces the correct answer:

In [11]:
answer(2.2, 44143124633, lambda:
       solve(id_ranges))

Puzzle  2.2:   .1350 seconds, answer 44143124633     ok

# [Day 3](https://adventofcode.com/2025/day/3) ChatGPT 5.1 Auto

ChatGPT gave a very brief analysis of the problem and then wrote a program that was designed to be called from the command line, using the `"__main__"` idiom. I told it "I don't want to run it like that, I want a function that I can pass the input text and have it return an int." and it produced this code (lacking comments or doc strings):

In [12]:
def total_output_joltage(input_text: str) -> int:
    def max_joltage_for_bank(bank: str) -> int:
        max_first = -1
        max_pair = 0
        for ch in bank:
            if not ch.isdigit():
                continue
            d = int(ch)
            if max_first != -1:
                pair = 10 * max_first + d
                if pair > max_pair:
                    max_pair = pair
            if d > max_first:
                max_first = d
        return max_pair

    total = 0
    for line in input_text.strip().splitlines():
        line = line.strip()
        if line:
            total += max_joltage_for_bank(line)
    return total

I verified this is correct:

In [13]:
banks = get_text(3)

answer(3.1, 17085, lambda:
       total_output_joltage(banks))

Puzzle  3.1:   .0044 seconds, answer 17085           ok

For Part 2 ChatGPT did well (and for some reason included comments and doc strings this time):

In [14]:
def total_output_joltage(input_text: str, k: int = 12) -> int:
    """
    Given the full puzzle input as a string, return the total maximum joltage
    when, for each line (bank), you must choose exactly k digits in order to
    form the largest possible number.
    """

    def max_joltage_for_bank(bank: str, k: int) -> int:
        # Extract digits from the line
        digits = [int(ch) for ch in bank if ch.isdigit()]
        n = len(digits)
        if k > n:
            raise ValueError(f"Bank has only {n} digits, cannot pick {k}.")

        # We will remove exactly n - k digits to maximize the resulting sequence
        removals = n - k
        stack = []

        for d in digits:
            # While we can still remove digits, and the last digit in the stack
            # is smaller than the current digit, pop it to make the number larger.
            while stack and removals > 0 and stack[-1] < d:
                stack.pop()
                removals -= 1
            stack.append(d)

        # If we still have removals left, remove from the end
        if removals > 0:
            stack = stack[:-removals]

        # Now stack should have length n - original_removals == k
        chosen = stack[:k]
        return int("".join(map(str, chosen)))

    total = 0
    for line in input_text.strip().splitlines():
        line = line.strip()
        if not line:
            continue
        total += max_joltage_for_bank(line, k)

    return total

In [15]:
answer(3.2, 169408143086082, lambda:
       total_output_joltage(banks))

Puzzle  3.2:   .0076 seconds, answer 169408143086082 ok

# [Day 4](https://adventofcode.com/2025/day/4): Gemini 3 Pro

Gemini produced a solution to Part 1 that is straightforward and efficient, although perhaps could use some abstraction (e.g. if they had a function to count neighbors, they wouldn't need the `continue`).

In [16]:
def count_accessible_rolls(grid_str):
    """
    Parses a grid string and counts paper rolls (@) that have
    fewer than 4 neighbors of the same type.
    """
    # Parse input into a list of lists (2D grid)
    # .strip() removes leading/trailing whitespace to handle clean input
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    accessible_count = 0
    
    # Directions for 8 neighbors: (row_offset, col_offset)
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    for r in range(rows):
        for c in range(cols):
            # We only care if the current spot is a paper roll
            if grid[r][c] != '@':
                continue
                
            neighbor_rolls = 0
            
            # Check all 8 neighbors
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                
                # Check boundaries
                if 0 <= nr < rows and 0 <= nc < cols:
                    if grid[nr][nc] == '@':
                        neighbor_rolls += 1
            
            # The Condition: fewer than 4 adjacent rolls
            if neighbor_rolls < 4:
                accessible_count += 1
                
    return accessible_count

Here I verify that the code does produce the correct answer:

In [17]:
grid = get_text(4)

answer(4.1, 1569, lambda:
       count_accessible_rolls(grid))

Puzzle  4.1:   .0171 seconds, answer 1569            ok

For Part 2, Gemini chose the "repeatedly remove acessible rolls" approach rather than the more efficient (and arguable more elegant) "remove a roll and check its neighbors" approach.

In [18]:
def count_total_removable_rolls(grid_str):
    """
    Parses a grid string and iteratively removes paper rolls (@) 
    that have fewer than 4 neighbors of the same type.
    Returns the total number of rolls removed.
    """
    # Parse input into a list of lists (2D grid)
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    total_removed_count = 0
    
    # Directions for 8 neighbors
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    while True:
        rolls_to_remove = []
        
        # 1. Identify all rolls that can be removed in this pass
        for r in range(rows):
            for c in range(cols):
                if grid[r][c] != '@':
                    continue
                    
                neighbor_rolls = 0
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols:
                        if grid[nr][nc] == '@':
                            neighbor_rolls += 1
                
                # Condition: fewer than 4 adjacent rolls
                if neighbor_rolls < 4:
                    rolls_to_remove.append((r, c))
        
        # 2. If nothing to remove, we are done
        if not rolls_to_remove:
            break
            
        # 3. Remove them (update grid and count)
        total_removed_count += len(rolls_to_remove)
        for r, c in rolls_to_remove:
            grid[r][c] = '.'  # Mark as empty space
            
    return total_removed_count

In [19]:
answer(4.2, 9280, lambda:
       count_total_removable_rolls(grid))

Puzzle  4.2:   .3960 seconds, answer 9280            ok

I asked Gemini "*Can you do it more efficiently*" and it replied

>The previous implementation scanned the entire grid repeatedly (`O(Rows * Cols * Iterations)`). I've replaced it with a Worklist/Queue algorithm that checks neighbors only when necessary. This reduces the complexity to be roughly proportional to the number of rolls, which is significantly faster for large grids.

This is the same efficiency gain that I realized in my solution.

In [20]:
def count_total_removable_rolls(grid_str):
    """
    Parses a grid string and iteratively removes paper rolls (@) 
    that have fewer than 4 neighbors of the same type.
    Returns the total number of rolls removed.
    
    Optimized to use a worklist (queue) approach instead of full grid scans.
    """
    # Parse input into a list of lists (2D grid)
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    total_removed_count = 0
    
    # Directions for 8 neighbors
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    def count_neighbors(r, c):
        count = 0
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == '@':
                    count += 1
        return count

    # 1. Initial scan: Find all rolls that are immediately removable
    queue = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '@':
                if count_neighbors(r, c) < 4:
                    queue.append((r, c))
    
    # 2. Process the queue
    # Using a list as a queue. Duplicates are handled by checking grid state.
    head = 0
    while head < len(queue):
        r, c = queue[head]
        head += 1
        
        # If already removed (e.g., added to queue multiple times), skip
        if grid[r][c] != '@':
            continue
            
        # Remove the roll
        grid[r][c] = '.'
        total_removed_count += 1
        
        # Check its neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                # If neighbor is a roll, check if it became removable
                if grid[nr][nc] == '@':
                    if count_neighbors(nr, nc) < 4:
                        queue.append((nr, nc))
            
    return total_removed_count

In [21]:
answer(4.2, 9280, lambda:
       count_total_removable_rolls(grid))

Puzzle  4.2:   .0937 seconds, answer 9280            ok

# [Day 5](https://adventofcode.com/2025/day/5): Claude Opus 4.5

Claude produces a straightforward program that solves Part 1 just fine and demonstrates good use of abstraction.

In [22]:
"""
Solution for the Fresh Ingredients puzzle.
Parses a database with fresh ingredient ID ranges and available ingredient IDs,
then counts how many available ingredients are fresh.
"""

def parse_input(input_text):
    """Parse the input into ranges and ingredient IDs."""
    parts = input_text.strip().split('\n\n')
    
    # Parse fresh ranges
    ranges = []
    for line in parts[0].strip().split('\n'):
        start, end = map(int, line.split('-'))
        ranges.append((start, end))
    
    # Parse available ingredient IDs
    ingredient_ids = []
    for line in parts[1].strip().split('\n'):
        ingredient_ids.append(int(line))
    
    return ranges, ingredient_ids


def is_fresh(ingredient_id, ranges):
    """Check if an ingredient ID falls within any fresh range."""
    for start, end in ranges:
        if start <= ingredient_id <= end:
            return True
    return False


def count_fresh_ingredients(ranges, ingredient_ids):
    """Count how many ingredient IDs are fresh."""
    count = 0
    for ingredient_id in ingredient_ids:
        if is_fresh(ingredient_id, ranges):
            count += 1
    return count


def solve(input_text):
    """Solve the puzzle and return the count of fresh ingredients."""
    ranges, ingredient_ids = parse_input(input_text)
    return count_fresh_ingredients(ranges, ingredient_ids)

Here I feed the input to `solve` and verify that the answer is correct:

In [23]:
input_text = get_text(5)

answer(5.1, 635, lambda:
       solve(input_text))

Puzzle  5.1:   .0058 seconds, answer 635             ok

For Part 2, Claude chose to sort ranges and then merge them. That's a reasonable approach.

In [24]:
"""
Solution for the Fresh Ingredients puzzle - Part 2.
Count all unique ingredient IDs that are considered fresh by any range.
"""

def parse_input(input_text):
    """Parse the input to get fresh ranges (ignore ingredient IDs section)."""
    parts = input_text.strip().split('\n\n')
    
    # Parse fresh ranges (only first section matters now)
    ranges = []
    for line in parts[0].strip().split('\n'):
        start, end = map(int, line.split('-'))
        ranges.append((start, end))
    
    return ranges


def merge_ranges(ranges):
    """Merge overlapping ranges to count unique IDs efficiently."""
    if not ranges:
        return []
    
    # Sort ranges by start value
    sorted_ranges = sorted(ranges)
    
    merged = [sorted_ranges[0]]
    
    for current_start, current_end in sorted_ranges[1:]:
        last_start, last_end = merged[-1]
        
        # Check if current range overlaps or is adjacent to the last merged range
        if current_start <= last_end + 1:
            # Merge by extending the end if needed
            merged[-1] = (last_start, max(last_end, current_end))
        else:
            # No overlap, add as new range
            merged.append((current_start, current_end))
    
    return merged


def count_fresh_ids(ranges):
    """Count total unique fresh ingredient IDs."""
    merged = merge_ranges(ranges)
    
    total = 0
    for start, end in merged:
        # Count is inclusive: end - start + 1
        total += end - start + 1
    
    return total


def solve(input_text):
    """Solve part 2 and return the count of all fresh ingredient IDs."""
    ranges = parse_input(input_text)
    return count_fresh_ids(ranges)

In [25]:
answer(5.2, 369761800782619, lambda:
       solve(input_text))

Puzzle  5.2:   .0002 seconds, answer 369761800782619 ok

# [Day 6](https://adventofcode.com/2025/day/6): ChatGPT 5.1 Auto

ChatGPT produced a program that is correct, but has poor abstraction, with one long 63-line function. (It also contains a pet peeve of mine: the pattern "`if some_boolean: True else: False`" can always be replaced with "`some_boolean`".)

In [26]:
import re
from functools import reduce
from operator import add, mul

def solve(lines):
    if not lines:
        return 0

    # Preserve internal spaces; pad all lines to same width
    width = max(len(line) for line in lines)
    grid = [line.rstrip('\n').ljust(width) for line in lines]
    h = len(grid)

    # Identify separator columns: columns that are all spaces
    sep = []
    for c in range(width):
        if all(grid[r][c] == ' ' for r in range(h)):
            sep.append(True)
        else:
            sep.append(False)

    # Find ranges of columns that form individual problems
    problem_ranges = []
    in_problem = False
    start = 0
    for c in range(width):
        if not sep[c]:
            if not in_problem:
                in_problem = True
                start = c
        else:
            if in_problem:
                in_problem = False
                problem_ranges.append((start, c - 1))
    if in_problem:
        problem_ranges.append((start, width - 1))

    grand_total = 0

    # Process each problem
    for c_start, c_end in problem_ranges:
        numbers = []

        # All rows except the last contain numbers (potentially)
        for r in range(h - 1):
            segment = grid[r][c_start:c_end + 1]
            for match in re.finditer(r'\d+', segment):
                numbers.append(int(match.group(0)))

        # Last row contains the operator (+ or *)
        bottom_segment = grid[h - 1][c_start:c_end + 1]
        if '+' in bottom_segment:
            op = add
        elif '*' in bottom_segment:
            op = mul
        else:
            # No valid operator found; skip this malformed problem
            continue

        if not numbers:
            continue

        # Fold numbers using the chosen operator
        result = reduce(op, numbers)
        grand_total += result

    return grand_total

Here I interface my input data to ChatGPT's function and verify it is correct:

In [27]:
lines = get_text(6).splitlines()

answer(6.1, 5877594983578, lambda:
       solve(lines))

Puzzle  6.1:   .0078 seconds, answer 5877594983578   ok

For Part 2 we get an even-longer function, with even more complications. For example in "Determine the operator"; the 14 lines could be replaced with `op = (add if '+' in segment else mul)`. However, the program is correct.

In [28]:
import re
from functools import reduce
from operator import add, mul

def solve(lines):
    if not lines:
        return 0

    # Normalize lines to a rectangle, preserving internal spaces
    width = max(len(line) for line in lines)
    grid = [line.rstrip('\n').ljust(width) for line in lines]
    h = len(grid)

    # A separator column is all spaces -> separates problems
    sep = []
    for c in range(width):
        if all(grid[r][c] == ' ' for r in range(h)):
            sep.append(True)
        else:
            sep.append(False)

    # Find contiguous ranges of non-separator columns (each is a problem)
    problem_ranges = []
    in_problem = False
    start = 0
    for c in range(width):
        if not sep[c]:
            if not in_problem:
                in_problem = True
                start = c
        else:
            if in_problem:
                in_problem = False
                problem_ranges.append((start, c - 1))
    if in_problem:
        problem_ranges.append((start, width - 1))

    grand_total = 0

    for c_start, c_end in problem_ranges:
        # Find the operator row for THIS problem slice:
        # bottom-most row with + or * in [c_start, c_end]
        op_row = None
        for r in range(h):
            segment = grid[r][c_start:c_end + 1]
            if '+' in segment or '*' in segment:
                op_row = r

        if op_row is None:
            # No operator for this vertical slice -> skip
            continue

        # Determine the operator
        op_segment = grid[op_row][c_start:c_end + 1]
        if '+' in op_segment and '*' in op_segment:
            # If both appear (shouldn't in valid input), choose the leftmost
            plus_idx = op_segment.find('+') if '+' in op_segment else float('inf')
            mul_idx = op_segment.find('*') if '*' in op_segment else float('inf')
            op = add if plus_idx < mul_idx else mul
        elif '+' in op_segment:
            op = add
        elif '*' in op_segment:
            op = mul
        else:
            # No operator actually present
            continue

        # Build numbers column-by-column (each column is one number),
        # reading digits from top to just above op_row
        numbers = []
        for c in range(c_start, c_end + 1):
            digits = []
            for r in range(op_row):
                ch = grid[r][c]
                if ch.isdigit():
                    digits.append(ch)
            if digits:
                num = int(''.join(digits))
                numbers.append(num)

        if not numbers:
            continue

        result = reduce(op, numbers)
        grand_total += result

    return grand_total

In [29]:
answer(6.2, 11159825706149, lambda:
       solve(lines))

Puzzle  6.2:   .0064 seconds, answer 11159825706149  ok

# Summary

The LLMs got all the problems right (with only a little extra prompting), and the programs are all reasonably efficient (the Day 2 programs could be faster). The readability of the code varies.

In [30]:
for d in sorted(answers):
    print(answers[d])
sum(a.secs for a in answers.values())

Puzzle  1.1:   .0013 seconds, answer 1182            ok
Puzzle  1.2:   .0024 seconds, answer 6907            ok
Puzzle  2.1:   .1263 seconds, answer 23560874270     ok
Puzzle  2.2:   .1350 seconds, answer 44143124633     ok
Puzzle  3.1:   .0044 seconds, answer 17085           ok
Puzzle  3.2:   .0076 seconds, answer 169408143086082 ok
Puzzle  4.1:   .0171 seconds, answer 1569            ok
Puzzle  4.2:   .0937 seconds, answer 9280            ok
Puzzle  5.1:   .0058 seconds, answer 635             ok
Puzzle  5.2:   .0002 seconds, answer 369761800782619 ok
Puzzle  6.1:   .0078 seconds, answer 5877594983578   ok
Puzzle  6.2:   .0064 seconds, answer 11159825706149  ok


0.40801501274108887